# Using Reinforcement Learning to Play a Fishing Game
*By Keegan Kochis and Landon Zweigle with love*

# TODO
* Python main loop 8

# Introduction
---
Ever since we have gotten into programming we have had a desire to do train a machine learning model to play a video game, and this term project seemed like a perfect excuse to finally do it, so we chose to train a reinforcement learning (RL) algorithm. When it came to deciding what to do RL on, we decided to use a [JavaFX](https://openjfx.io/) game Landon made a few years ago; a very simple fishing minigame where the goal is to put a bobber over a fish for a certain amount of time. This game had everything we were looking for: an evironment which is rather simple (but not too simple) and an easy way to communicate with it.



The game as described earlier is a simple fishing minigame. It is actually a clone of minigame in a very popular indie game [Stardew Valley](https://www.stardewvalley.net/). Here is an indepth [description](https://stardewvalleywiki.com/Fishing#Overview_.26_Controls) of the fishing minigame. The image below is a screenshot from Stardew Valley's fishing minigame.

![The game](https://www.carlsguides.com/stardewvalley/pictures/fishing/catch-fish.jpg)


The mini-game is a simple objective of keeping a capture area (aka "the bobber") behind a randomly moving capture-point (aka "the fish") for long enough to win the game. In the image, the capture-point is the fish and the area you move (the bobber) is the green bar. You can change the velocity of the bobber by accelerating it upwards (holding down the space bar) or letting gravity pull the bobber downwards (releasing the space bar). The capture time left is indicated by the "capture bar", the large green bar on the left. The goal is to raise the capture bar to the top of the mini-game window. Raising it to the top means you have successfully caught the fish. Letting the capture bar decrease to the bottom means the fish has escaped. In the original game each time the minigame starts the fish's difficulty is randomized. However, to simplify things we have chosen to leave the difficulty of all the fish at a fixed value.




Because the game was built in Java and our RL algorithm would be implemented in Python, we needed a way to communicate between Python and Java. For this we decided to use socket programming. Further, because of the random movement of the fish, we knew we had to use a neural network to approximate the Q-table.

# Methods
---
Naturally, we needed a neural network class for the solution to our problem. For this, we made use of some code provided to us in lectures [17](https://nbviewer.jupyter.org/url/www.cs.colostate.edu/~cs445/notebooks/17%20Reinforcement%20Learning%20with%20Neural%20Network%20as%20Q%20Function.ipynb) and [18](https://nbviewer.jupyter.org/url/www.cs.colostate.edu/~cs445/notebooks/18.1%20Reinforcement%20Learning%20to%20Control%20a%20Marble.ipynb), the neural network class and it's sibling optimizer class. We regularly visited and referenced these lectures for information regarding reinforcement learning algorithms and implementation details. Everything else was written by us.

## Steps taken
#### Java-Python Communications
Because Landon wrote the game, he implemented the communication between Java and Python. There were several options he could have gone with but he ended up writting a simple java socket server which python would recieve the games state from, and map actions to, all while keeping the game loop synced to the RL algorithm loops.

#### Convenient RL Neural Network
Keegan integrated the neural network class with a few modifications. He implemented a trivial `EpsilonGreedyUse` method which had access to the nueral networks `self` variable. The `EpsilonGreedyUse` method had access to the `self` variable in order to take a more object oriented approach. This allowed the function to reference `self.use` and `self.epsilon`. He also implemented a template `getReinforcement` method which was tuned later to increase algorithm performance (accuracy). We will discuss the tuning in more detail in a later section.

#### Main loop
Landon worked on the "main loop" of our program. The main loop deals with the main processes in the RL algorithm; getting the data for each trial, training the model, saving the results, etc... We will touch on the main loop in more detail later.

#### Secondary Code
For this project to work properly a lot of secondary code was neccessary. Some of the functions that were neccesary include a function which can easily create a csv of experiments to be run, and a function which actually executes the experiments. At this point in development, both of us (your humble and handsome protagonists, Keegan and Landon) contributed to this code.

#### Fine tunning
To experiment with producing a functional model both of us would frequently adjust what Java reports as the state, and what the reinforcement/reward is defined as.


## The Main Loop.
For our implementation we had two proccesses communicating synchronously. It took considerable effort to make Java work syncronously with Python. In this section we will discuss the two proccess.

### The Java Loop:
In order to get our stand alone Java game to work with Python we needed to make several changes. It was most fitting that Landon undertook this task because he was most familiar with the game, being the person who created it. To begin with, there were several bugs in the game that made the fish inconsitent. To be specific, much of the physics did not take into account the frame rate at which the game is running at. This caused the fish to zoom around (at the speed of sound, wow!) when the game was operating at very high framerates. We needed to fix this so the physics would be consistant across all framerates. This would ensure that the game always runs the same regardless of the hardware it is being run on. Next, we needed a way of syncing the frame rate up with Python. To do this, we added a simple condition check at the begining of the frame loop in Java. This condition check would only allow the loop to continue with the rest of the code if Python gave it a signal to do so. The final major fix we needed to add was a way of easily communicating to Python what the current state of the game was. To do this we added a method, in the Java code, which would grab all the information from the game, and convert it into a string. After this method created the string containing the game's information it would send the string to Python.

Now that these changes were complete, we could implement the Java side of the main loop:

1. The loop conditional recieves an integer from Python, and if it is not zero the loop body will be executed. If the integer is zero the loop ends and the game stops. This integer is how Python reports the state of the loops to Java (not to be confused with the state of the game reported _by_ Java). This integer basically tells Java to either quit the loop or start a new game (described below).

3. Inside the loop, Java uses the same integer (loop-state) as used in the loop conditional to check an if statement. If the condition is true Java resets the game to a random state which is immediatly reported to Python. This is done so that when Java starts a new trial it reports the inital state to Python.

4.  Outside of the if statement, Java recieves another integer which will relate to the action Python desires to take. Java will then perform the requested action and process the next frame.

5. Next, Java will wait until the last requested frame has fully been rendered to the screen and proccessed by Python.

6. After the frame has been rendered, Java will send an integer to Python reporting that the frame has successfuly been rendered. This is just a sanity check so Python can verify that Java is still running properly.

7. Finally, Java will grab the current state, convert it to a string, and then send it to Python to proccess. After this step Java returns to the loop-conditional.

### The Python Loop:
The Python code that deals with communications is nearly identical to Java so we will be focusing on and highlighting the differences between the two loops.

1. The loop conditional for python checks that the current frame is less than the total amount of frame to be proccessed (`FramePerTrial` * `nTrials`).

2. Next, Python reports to Java the state of the loop.

3. If the state of the loop is to start a new trial, Python grabs a new state from Java as the initial state and then uses the `EpsilonGreedyUse` function to generate an initial action.

4. If Python did not execute the last described step (step 3, determining an initial action for a new trial) it will instead use the previously determined action (this previously determined action is described in step 7 and is taken from the previous frame). Python will then report the desired action to Java.

5. Next Python will recieve an int from Java checking to ensure that Java is still operatiing as desired, and raise an error if the int is not expected.

6. After Python checks the health of Java, Python will recieve the game state from Java in the form of a string. Python will then tokenize this String to get a list of integers.

7. After recieving the current state, Python will determine the reinforcement using the `getReinforcement` method and decide the next action to take using `EpsilonGreedyUse`.

8. Using the previous state and action, Python will create a state-action pair. Python will then append this state-action pair to the end of the `X` array. Similary, the _current_ reinforcement is appended to the `R` array. The `X` and `R` arrays essentially give a history of past state-action pairs and the reinforcement for the next state created as a result.

9. Python now checks whether or not to end this trial and start a new one (`currentFrame % framesPerTrial == 0`). If a new trial needs to be started, Python will do several things:
    1. Proccess the data created over all step 8s (one per frame) for this trial.
        * Take a sum over all past reinforcements for the given weight them using `gamma`.
    2. Train the neural network again with the newly proccessed data.
    3. After training, Python will decay Epsilon to make random movements less likely.
    
10. If there are more trials to be run then Python will continue looping, otherwise Python will:
    1. Tell Java to terminate
    2. Create several output files and proceed to exit. These output files are discussed in greater detail later on.

In regards to the loop, we had a significant choice to make. Should we reset the game after each trial or not? As shown above you can see that we decided to reset the game after each trial. We went with this approach for a couple of reasons, the main of which is to expose the algorithm to as many states as possible. We noticed that it took several hundred frames for the game's state to significantly change. Consider the following graph:

![Results without reseting](https://i.imgur.com/axGvkhb.png)

This graph is the result of training and _not_ reseting the game after each trial. Upon first inspection it would appear as if the results are decently good. However, upon closer inspection you will notice that these results show that bad performance happens as often as good performance. We can spot points of poor performance when the orange line reaches a value of 2 or above. A value of 2 or higher means that the "bobber" is moving away from the fish. We can spot points of good performance on the graph when the orange line reaches a value of 1 or lower. A value of 1 or lower denotes that the "bobber" is colliding with the fish. Values between 1 and 2 also signify decent performance. Values within this range mean that the "bobber" is moving towards the fish. The reason for the poor performance shown in the graph above is because of the fact that game takes a really long time for any significant change in state. The low (good) scores are low purely by coincidance. It is also worth noting that the amount of frames per trial (5 frames per trial, very low) also makes it appear as if the model is learning.

The graph below was also the result of _not_ reseting the game after each trial. The difference in the graph below and the graph above is that the graph below is the result of using a larger amount of frames per trial (20). Notice how it would appear that this model performs much worse. It is simply because the mean reinforcement was over a much larger range and, as such, tended to be more tightly packed. We reached the conclusion that not reseting the game after each trial introduced bias into our model.

![Results without reseting V2](https://i.imgur.com/igJbWLo.png)


## Reinforcement: A cost.

The backbone of reinforcement learning is assining a value to certain state. Some states are more desirable than others, and as such you need a way of telling the Q-table this. In our case, it is quite simple. We want the bobber to be as close to the fish as possible. Furthermore, because the fish moves randomly, we would like the bobbers velocity to match that of the fish. In essence, we are trying to **minimize** the distance between the bobber and the fish, thus our reinforcement is a **cost** we would like to minimize. Considering these simple facts, this is how we came up with our reinforcement:

Because the bobber and the fish both have a veloctity, and hence have a position, the reinforcement is a little more complex than just how far the bobber is from the fish. Throughout development we kept an ideology in mind such that there are technically 4 types of states:
* `Best Colliding`: The bobber is behind the fish and their velocities are in the same direction (equal normalized velocity).

* `Simple Colliding`: The bobber is behind the fish, but they have differing normalized velocities.

* `Seperate`: The bobber is not behind the fish, but they have equivalent normalized velocities.

* `Poor Seperate`: The bobber is not behind the fish and their normalized velocites are inequivalant.

In general we knew we wanted to minimize the reinforcement, and as such we decided that the reinforcement for each state was as such: `Best Colliding` < `Simple Colliding` < `Seperate` < `Poor Seperate`.

We first began with a simple reinforcement based off of this idea that follows the above definition. `Best Colliding` returned 0, `Simple Colliding` returned 1, `Seperate` returned 2, and `Poor Seperate` returned 3.

After discussing our problem we decided that our reinforcement should also encode the fish-bobber distance. To do this we realized that we couldn't just return the distance between the two. For instance, we considered an example where the fish and bobber are _not_ colliding but _are_ moving in the same direction. In this example we took the fish's position as greater than the bobber's position (the fish is above the bobber) and took both of their velocities as upward (they're both moving upwards). This means that the subsequent state is `Seperate`. If we took their velocites as downward, then the subsequent state is `Poor Seperate`. These two examples illustrate that if we were to just return the distance of the fish and bobber then the ideology of state described above would be lost. Because of this we only encode their distance when the state is `Seperate`. However, this methodology introduced another problem. `Seperate`'s reinforcement value according to the ideology should be between 1 and 3, whereas the distance could be anywhere from 0 to 1000. This meant we needed to have a standardized distance between 0 and 1, so we simply take the percentage of the distance over the total possible distance (`distance/max_range`). This didn't entirely solve our problem though, as the new reinforcement for `Seperate` would be between 0 and 1, when it should be between 1 and 3, so we simply added 1 to the percentage. This gave us a reinforcement function defined as:

    if `Best Colliding`:
        return 0
    elif `Simple Colliding`:
        return 1
    elif `Seperate`:
        return (distance/max_range) + 1
    elif `Poor Seperate`:
        return 3

We will discuss the downside of this strategy in the results Section.

## Training

For training we made two flexible sccripts; `createExperiments` and `run`. `createExperiments` simply creates a csv with several experiments which will then be parsed in `run` and passed to the main Python script, `AutoFisher`, then executed. `run`, as mentioned before, parses each experiment, such that for every experiment it will launch an instance of the java game then run our main reinforcement script `AutoFisher` passing in the training arguments. After each experiment `AutoFisher` creates a directory with 4 files in it; `ActionState.csv`, `DQN.dump`, `meanReinByTrial.png` and `results.csv`.

* `ActionState.csv` is a record of the last-n state-action pairs. This is useful for knowing what action is made in several states.
* `DQN.dump` is the trained Deep-Q-Network (DQN) which was pickled. We pickled the network so we could potentially use it later for further training or to play a full game with.
* `meanReinByTrial.png` is a graph displaying the mean reinforcement per trial. There is also a 20-trial average overlayed in orange. Examples of the graph can be found below and above.
* `results.csv` displays the parameters for the experiment (I.E. network architecture, number of frames per trial, etc).

We will discuss further in the results section exactly what we used for parameters in the experiments we ran, but over the course of 3 weeks we ran somewhere between one to two hundred experiments using these classes.

![fish](https://i.imgur.com/S0nNkDT.png)

# Results
----------
As explained earlier, over a three or four week period we ran somewhere in the range of 100 to 200 experiments. For running experiments we had a general philosophy: run whatever and iterate. We wanted to mostly experiment with small changes in code (such as what we report as the game's state), and as such we had a fixed set of experiments we ran. We only iterated on nueral network parameters a few times, testing mainly different numbers of frames per trial (5 - 200) and hidden layer architecturs. We hypothesized that increasing the number of trials would only lead to better results so we didn't test many different values. We would change these parameters to see how much they affected our results but in general most of our testing had little to do with parameters. For the first couple sets of experiments we tested heavily with what was reported in the game's state. For example, our very first experiment set was done with a very simple state. All that was reported was the difference in position (deltaP) and the difference in velocity (deltaV). Here are most of the state tracking schemes we ended up testing (in chronological order):

* deltaP, deltaV
* bobber-pos, fish-pos, bobber-vel, fish-vel
* bobber-pos, fish-pos, normalized-bobber-vel, normalized-fish-vel
* normalized-bobber-pos, normalized-fish-pos, normalized-bobber-vel, normalized-fish-vel
* bobber-pos, fish-pos, bobber-vel, fish-vel (we tested this scheme a second time)

As you can see we ended using the most verbose state we could, the final state scheme we tested. We decided that we can always reduce to any of the other formats if we needed to do so. However, we are currently not doing any proccessing of the state (except for where needed to get reinfocement, see ` Reinforcement: A cost` above). Here are a few mean-reinforcement graphs from each state format. You will notice that some of the graphs are critically different. Unfortunately we had some other code changes which affected the graphs output.

state: detlaP, deltaV.
![deltaP, deltaV](https://i.imgur.com/0wbi6Wt.png?1)

state: bobber-pos, fish-pos, bobber-vel, fish-vel.
![bobber-pos, fish-pos, bobber-vel, fish-vel](https://i.imgur.com/MiQEYcY.png)

state: bobber-pos, fish-pos, normalized-bobber-vel, normalized-fish-vel
![bobber-pos, fish-pos, normalized-bobber-vel, normalized-fish-vel](https://i.imgur.com/m5fJJV1.png)

state: normalized-bobber-pos, normalized-fish-pos, normalized-bobber-vel, normalized-fish-vel.
![normalized-bobber-pos, normalized-fish-pos, normalized-bobber-vel, normalized-fish-vel.](https://i.imgur.com/b9hFoLp.png)

state: bobber-pos, fish-pos, bobber-vel, fish-vel
![bobber-pos, fish-pos, bobber-vel, fish-vel](https://i.imgur.com/wAE4RpB.png)

As you can see,  our results were never that great. To capture the fish the reinforcement would be need to be between zero and one for a long duration of time (again, see ` Reinforcement: A cost` from above). The only graph that does have substantial reinforcement in our desired range is the very first graph. However, we had a _slightly_ erronous reinforcement function for that method. Further, as you can see, the first three graphs appear to indicate random movement of the bobber (given how some of the blue lines are just a solid block from start to end). This led us to discover a slightly embarrsing bug in our code. We were never decaying epsilon! We totally forgot that we moved Epsilon to be a class variable of the neural network, so the whole time we were decaying _the wrong epsilon_. Once we discovered this and fixed it, we saw much better results which seemed to indicated some level of "intelligence" in the bobber's movement. However, as you can clearly see, the average reinforcement was _still_ unfortunately not dipping below 1, like we hoped.

### Neural Network parameters

As previously mentioned, we did of course run more than one experiment per attempted state. In total, we had about 26 experiments (13 each person) per run. In general, we had 6 parameters which we were able to change from experiment to experiment. However, we only ended up using 4 of these as we figured that the other 2 parameters only lead to relatively minor changes in the results. The 4 parameters we iterated on were:
* Frames per Trial
* Number of Trials
* Hidden Layer Architecture
* Number of Epochs

We decided on a couple of values to try for each of these parameters. For frames per trial we had ended up using a pretty generous range between three values, 25, 50, and 100. Initially we experimented with values as low as 5 frames per trial and as high as 500 frames per trial but ended up settling with our happy medium. For Frames per Trial we were limited to using larger values because of the fact that we were reseting after each Trial. This is because if we did not include enough frames in a trial, some initial states entirely prevented the bobber from reaching the fish due to a lack of time. Number of trials was directly related to frames per trial. We simply had a total number of frames we wanted to keep constant between experiments, so we divided the target number of frames by the frames per trial to get the total amount of trials to run. We had several targets; 200,000, 100,000, and 50,000 frames.

For Hidden Layer Architecture, as usual it is mostly guessing. For this we mostly did random architectures, with between 1 and 3 hidden layers and between 20 to 100 neurons per layer. We experimented with bottle neck architectures as well as u shaped architectures.

For number of Epochs we randomly chose from one of three values, including 100, 200, and 300 epochs.

The other parameters that we chose to keep constant (but had the option to change) were as follows:
* Learning Rate
* Gamma

The reason we kept these constant was to reduce the total amount of experiments we had to run. It was already taking about a day to run each set of experiments, we didn't want to increase that to two or three days by enumerating many possible values for each parameter. We also figured that the values we had set would generally produce good results.

# Conclusion
---
There are several reasons we have considered as to why our algorithm was performing so poorly. The first and most obvious error is human error. It is entirely possible that we have one or more bugs in our code. While we do not think this is the case, it is quite possible that while contributing to the same code something got flipped around, perhaps accidentally in a merge conflict, and has not yet been discovered. It is (atleast in our experience) pretty hard to guarantee the correctness of machine learning code, especially because it is so much easier to just blame bad data.

The second possibility we have considered has to do with the game itself, specifically the fish's movement. As mentioned earlier, the fish's movement is random. As is obvious, it is impossible to predict something that is random. You can generalize randomness (as we are attempting to do here), but it is likely that the fish's random movement is just too hard for our algorithm to deal with. If we were to do this again, we would have liked to feed the last n frames into a convolutional nueral network to determine the state. We figure this would allow the model to "foresee" when the next change in velocity might be, further generalising the fish's movement. The problem with this approach is that we likely do not have the knoweledge to do something this advanced.

With all this being said, we're not satisfied with our results. We wanted and half-expected to have a model which was trained sufficiently well enough to play a full game and catch fish. Alas this is not the case. We're underwhelmed with our results because it seems that none of our models actually learned. We wouldv'e like to see the average reinforcment decrease over time but this is not the case. As depicted in our many graphs, the reinforcement for a given experiment essentially stayed at a fixed value for the duration of the experiment.

# INCLUDE THIS LANDON:

Even if the bobber moved towards the fish, the fish could move away, thus keeping the distance around constant and therefore lowering the insentive of moving towards the fish.

In [9]:
import io
from nbformat import current
import glob
name="./Kochis-Zweigle"
nbfile = glob.glob(name+'.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file ./Kochis-Zweigle.ipynb is 4208
